<a href="https://colab.research.google.com/github/Marya204/Recommendation-System-for-job-offers/blob/Processing/Traitement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **downloading dependencies**

In [ ]:
!pip install requests
!pip install beautifulsoup4

In [ ]:
!pip install selenium
!apt-get update
!apt-get remove -y chromium-browser
!apt-get autoremove -y
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -y --fix-broken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 34.0 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [343 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,130 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy In

In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install webdriver-manager

# **Emploima Scraper**

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import pandas as pd
import re

# Import webdriver_manager to handle Chromedriver installation
from webdriver_manager.chrome import ChromeDriverManager

def init_driver():
    """Initialise le driver Chrome"""
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
    chrome_options.binary_location = '/usr/bin/google-chrome' # Explicitly set Google Chrome binary location

    # Use ChromeDriverManager to install and get the latest Chromedriver path
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def get_job_links(driver, max_pages=20):
    """Récupère tous les liens d'offres SANS time.sleep"""
    all_links = set()

    for page in range(0, max_pages):
        try:
            url = f"https://www.emploi.ma/recherche-jobs-maroc?page={page}"
            print(f"📄 Scraping page {page + 1}/{max_pages}...")
            driver.get(url)

            # Attendre intelligemment le chargement
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "card-job"))
            )

            offer_links = driver.find_elements(By.XPATH, "//a[contains(@href, '/offre-emploi-maroc/')]")

            page_links = 0
            for link in offer_links:
                try:
                    href = link.get_attribute('href')
                    if href and '/offre-emploi-maroc/' in href:
                        all_links.add(href)
                        page_links += 1
                except:
                    continue

            print(f"🔗 Found {page_links} new links on page {page + 1} (Total: {len(all_links)})")

        except Exception as e:
            print(f"❌ Error on page {page}: {e}")
            continue

    return list(all_links)

def normalize_education_level(education_text):
    """Normalise le niveau d'études"""
    if not education_text or education_text == "N/A":
        return "Non spécifié"

    education_text = education_text.lower()

    # Détection des niveaux d'études
    if 'bac+5' in education_text or 'bac +5' in education_text or 'master' in education_text or 'ingénieur' in education_text:
        return "BAC+5"
    elif 'bac+4' in education_text or 'bac +4' in education_text or 'licence' in education_text:
        return "BAC+4"
    elif 'bac+3' in education_text or 'bac +3' in education_text:
        return "BAC+3"
    elif 'bac+2' in education_text or 'bac +2' in education_text:
        return "BAC+2"
    elif 'bac' in education_text and '+' not in education_text:
        return "BAC"
    elif 'doctorat' in education_text or 'phd' in education_text:
        return "DOCTORAT"

    # Si c'est une liste de niveaux
    if any(pattern in education_text for pattern in ['bac+', 'bac +']):
        levels_found = []
        for level in ['bac+5', 'bac+4', 'bac+3', 'bac+2', 'bac']:
            if level in education_text:
                levels_found.append(level.upper())

        if levels_found:
            return " - ".join(levels_found)

    return "Non spécifié"

def normalize_contract_type(contract_text):
    """Normalise le type de contrat"""
    if not contract_text or contract_text == "N/A":
        return "Non spécifié"

    contract_text = contract_text.lower()

    # Détection directe des contrats
    if 'cdi' in contract_text:
        return "CDI"
    elif 'cdd' in contract_text:
        return "CDD"
    elif 'stage' in contract_text:
        return "STAGE"
    elif 'intérim' in contract_text or 'interim' in contract_text:
        return "INTÉRIM"
    elif 'freelance' in contract_text:
        return "FREELANCE"
    elif 'anapec' in contract_text:
        return "ANAPEC"
    elif 'alternance' in contract_text:
        return "ALTERNANCE"
    elif 'temps partiel' in contract_text or 'part time' in contract_text:
        return "TEMPS PARTIEL"
    elif 'mission' in contract_text:
        return "MISSION"

    return "Non spécifié"

def normalize_experience(experience_text):
    """Normalise l'expérience requise"""
    if not experience_text or experience_text == "N/A":
        return "Non spécifié"

    experience_text = experience_text.lower()

    # Détection directe
    if 'débutant' in experience_text or 'junior' in experience_text or '0 an' in experience_text:
        return "Débutant"
    elif '1 an' in experience_text or '2 ans' in experience_text:
        return "1-2 ans"
    elif '3 ans' in experience_text or '4 ans' in experience_text or '5 ans' in experience_text or 'entre 2 ans et 5 ans' in experience_text:
        return "2-5 ans"
    elif '6 ans' in experience_text or '7 ans' in experience_text or '8 ans' in experience_text or '9 ans' in experience_text or '10 ans' in experience_text or 'entre 5 ans et 10 ans' in experience_text:
        return "5-10 ans"
    elif 'plus de 10' in experience_text or '10+' in experience_text or 'senior' in experience_text:
        return "10+ ans"

    return "Non spécifié"

def scrape_job_details(driver, url):
    """Scrape les détails d'une offre SANS time.sleep"""
    try:
        print(f"    🔍 Scraping: {url}")
        driver.get(url)

        # Attendre intelligemment
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        # Title
        title = "N/A"
        try:
            title_elem = driver.find_element(By.TAG_NAME, "h1")
            title = title_elem.text.strip()
        except:
            pass

        # Company
        company = "N/A"
        try:
            company_elems = driver.find_elements(By.XPATH, "//a[contains(@href, '/recruteur/')]")
            for elem in company_elems:
                company_text = elem.text.strip()
                if company_text and len(company_text) > 2 and company_text != "Voir toutes les offres":
                    company = company_text
                    break
        except:
            pass

        # MÉTHODE INTELLIGENTE : Analyser TOUS les éléments
        education_level = "Non spécifié"
        contract = "Non spécifié"
        experience = "Non spécifié"
        city = "Non spécifiée"

        try:
            # Récupérer tous les éléments de la liste
            list_items = driver.find_elements(By.XPATH, "//*[@id='main-content']/div[2]/div[1]/div[1]/div[1]/div[1]/div/ul/li")

            for item in list_items:
                text = item.text.strip().lower()

                # Détection du niveau d'études
                if any(edu_keyword in text for edu_keyword in ['bac+', 'bac +', 'diplôme', 'niveau d', 'formation']):
                    if 'expérience' not in text:  # Éviter les confusions
                        education_level = item.find_element(By.TAG_NAME, "span").text.strip()

                # Détection du contrat
                elif any(contract_keyword in text for contract_keyword in ['cdi', 'cdd', 'stage', 'intérim', 'contrat', 'freelance']):
                    contract = item.find_element(By.TAG_NAME, "span").text.strip()

                # Détection de l'expérience
                elif any(exp_keyword in text for exp_keyword in ['expérience', 'débutant', 'junior', 'senior', 'ans']):
                    if 'bac' not in text:  # Éviter les confusions
                        experience = item.find_element(By.TAG_NAME, "span").text.strip()

        except Exception as e:
            print(f"    ⚠️  Méthode intelligente échouée: {e}")

        # VILLE - XPATH EXACT
        try:
            city_elem = driver.find_element(By.XPATH, "//*[@id='main-content']/div[2]/div[1]/div[1]/div[1]/div[1]/div/ul/li[2]/span")
            city = city_elem.text.strip()
        except:
            # Fallback pour la ville
            try:
                location_elems = driver.find_elements(By.XPATH, "//li[contains(., 'Région') or contains(., 'Ville') or contains(., 'Lieu')]")
                for elem in location_elems:
                    location_text = elem.text
                    if ":" in location_text:
                        city = location_text.split(":")[1].strip()
                        break
            except:
                pass

        # Normalisation
        education_level = normalize_education_level(education_level)
        contract = normalize_contract_type(contract)
        experience = normalize_experience(experience)

        print(f"    📌 Titre: {title[:50]}...")
        print(f"    🏢 Entreprise: {company}")
        print(f"    🎓 Niveau: {education_level}")
        print(f"    📝 Contrat: {contract}")
        print(f"    💼 Expérience: {experience}")
        print(f"    📍 Ville: {city}")

        # Publication date
        publication_date = "Non spécifiée"
        try:
            date_elem = driver.find_element(By.XPATH, "//*[@id='main-content']/div[1]/div/div[2]/p")
            date_text = date_elem.text.strip()
            date_match = re.search(r'\d{2}\.\d{2}\.\d{4}', date_text)
            if date_match:
                publication_date = date_match.group()
            print(f"    📅 Date: {publication_date}")
        except:
            pass

        # Description
        description = "Non disponible"
        try:
            desc_elems = driver.find_elements(By.XPATH, "//div[contains(@class, 'description')] | //div[contains(@class, 'content')] | //article")
            for elem in desc_elems:
                desc_text = elem.text.strip()
                if desc_text and len(desc_text) > 50:
                    description = re.sub(r'\s+', ' ', desc_text)
                    break
        except:
            pass

        job_data = {
            "site": "Emploi.ma",
            "title": title,
            "company": company,
            "city": city,
            "education_level": education_level,
            "contract": contract,
            "experience": experience,
            "publication_date": publication_date,
            "description": description[:500] + "..." if len(description) > 500 else description,
            "url": url
        }

        return job_data

    except Exception as e:
        print(f"❌ Error scraping {url}: {e}")
        return None

def scrape_emploi_ma():
    """Fonction principale de scraping TOUTES les offres"""
    print("🚀 Starting Emploi.ma scraper (TOUTES les offres)...\n")

    driver = init_driver()
    all_jobs = []

    try:
        # Get all job links from 20 pages
        job_links = get_job_links(driver, max_pages=20)
        print(f"\n🔗 Total unique links found: {len(job_links)}\n")

        # Scrape CHAQUE job (pas seulement 10)
        total_links = len(job_links)
        for i, link in enumerate(job_links, 1):
            print(f"[{i}/{total_links}]")
            job_data = scrape_job_details(driver, link)

            if job_data:
                all_jobs.append(job_data)
                print(f"  ✅ {job_data['title'][:40]}...")

            # Afficher la progression
            if i % 10 == 0:
                print(f"\n📊 Progression: {i}/{total_links} offres traitées ({i/total_links*100:.1f}%)\n")

    finally:
        driver.quit()

    return all_jobs

# EXÉCUTION
if __name__ == "__main__":
    jobs = scrape_emploi_ma()

    if jobs:
        df = pd.DataFrame(jobs)

        # Renommer les colonnes en français
        df = df.rename(columns={
            "site": "site",
            "title": "intitulé_poste",
            "company": "entreprise",
            "city": "ville",
            "education_level": "niveau_études",
            "contract": "type_contrat",
            "experience": "expérience",
            "publication_date": "publication_date",
            "description": "description",
            "url": "lien_offre"
        })

        filename = "emploi_ma_toutes_offres.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')

        print(f"\n🎉 COLLECTE TERMINÉE!")
        print(f"📁 Fichier sauvegardé: {filename}")
        print(f"📊 Total offres collectées: {len(df)}")

        # Rapport détaillé
        print(f"\n📈 RAPPORT STATISTIQUE:")
        print(f"• Villes: {df['ville'].value_counts().head(10).to_dict()}")
        print(f"• Types de contrat: {df['type_contrat'].value_counts().to_dict()}")
        print(f"• Niveaux d'études: {df['niveau_études'].value_counts().to_dict()}")
        print(f"• Expériences: {df['expérience'].value_counts().to_dict()}")

        # Aperçu des données
        print(f"\n👀 APERÇU DES DONNÉES:")
        print(df[['intitulé_poste', 'entreprise', 'ville', 'niveau_études', 'type_contrat']].head(15))
    else:
        print("❌ No jobs scraped!")

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
    📝 Contrat: CDI
    💼 Expérience: Débutant
    📍 Ville: Agadir - Casablanca-Mohammedia - Dakhla - Errachidia - Fès - Laâyoune - Marrakech-Safi - Rabat-Salé-Kénitra - Settat - Tanger-Tétouan-Al Hoceïma - International
    📅 Date: 07.11.2025
  ✅ Factory Director (Garment) - (Expatriati...
[22/500]
    🔍 Scraping: https://www.emploi.ma/offre-emploi-maroc/assistante-administrative-commerciale-ain-harrouda-casablana-mohammedia-9076370
    📌 Titre: Assistante Administrative et Commerciale - Ain Har...
    🏢 Entreprise: JOB CONNECT
    🎓 Niveau: BAC+3
    📝 Contrat: CDD
    💼 Expérience: Débutant
    📍 Ville: Casablanca-Mohammedia
    📅 Date: 18.10.2025
  ✅ Assistante Administrative et Commerciale...
[23/500]
    🔍 Scraping: https://www.emploi.ma/offre-emploi-maroc/charge-portefeuille-hf-oujda-9103648
    📌 Titre: Chargé de Portefeuille H/F - Oujda...
    🏢 Entreprise: FONDATION ARRAWAJ
    🎓 Niveau: BAC+5
    📝 

In [ ]:
df.head()

,site,intitulé_poste,entreprise,ville,niveau_études,type_contrat,expérience,publication_date,description,lien_offre
0,Emploi.ma,Acheteur - Nouaceur,TECTRA BERRECHID,Casablanca-Mohammedia,BAC+5,INTÉRIM,Débutant,12.11.2025,Résumé du poste Achats Casablanca-Mohammedia E...,https://www.emploi.ma/offre-emploi-maroc/achet...
1,Emploi.ma,Responsable des Ressources Humaines - Mohammedia,BRIQUETERIE JBEL ANNOUR,Casablanca-Mohammedia,BAC+5,CDI,Débutant,22.10.2025,Résumé du poste Métiers du BTP Casablanca-Moha...,https://www.emploi.ma/offre-emploi-maroc/respo...
2,Emploi.ma,"Formateur Vacataire en ""Exploiter les instrume...",INSTITUT DE FORMATION AUX MÉTIERS DU BTP,Fès,BAC+5,CDD,Débutant,11.11.2025,Résumé du poste Métiers du BTP Fès Expérience ...,https://www.emploi.ma/offre-emploi-maroc/forma...
3,Emploi.ma,Valet de Chambre - Marrakech,MAROC FORCE EMPLOI,Marrakech-Safi,BAC+5,INTÉRIM,Débutant,05.11.2025,"Résumé du poste Tourisme, hôtellerie, restaura...",https://www.emploi.ma/offre-emploi-maroc/valet...
4,Emploi.ma,Responsable Marketing et Communication - Casab...,CALLIOPE,Casablanca-Mohammedia,BAC+5,CDI,Débutant,10.11.2025,"Résumé du poste Marketing, communication Casab...",https://www.emploi.ma/offre-emploi-maroc/respo...


In [ ]:
df.tail()

,site,intitulé_poste,entreprise,ville,niveau_études,type_contrat,expérience,publication_date,description,lien_offre
495,Emploi.ma,"Aide Comptable ""Expérience dans la Gestion de ...",SOLU-PERFORMANCE,Tanger-Tétouan-Al Hoceïma,BAC+3,CDI,Débutant,16.10.2025,"Résumé du poste Gestion, comptabilité, finance...",https://www.emploi.ma/offre-emploi-maroc/aide-...
496,Emploi.ma,Chargé de Portefeuille H/F - Tanger,FONDATION ARRAWAJ,Tanger-Tétouan-Al Hoceïma,BAC+5,CDI,Débutant,05.11.2025,"Résumé du poste Commercial, vente Gestion, com...",https://www.emploi.ma/offre-emploi-maroc/charg...
497,Emploi.ma,Stagiaire Développeur(se) Web et Mobile (PHP/L...,VISION BUSINESS CONSULTING,Casablanca-Mohammedia,BAC+5,STAGE,Débutant,30.10.2025,"Résumé du poste Informatique, nouvelles techno...",https://www.emploi.ma/offre-emploi-maroc/stagi...
498,Emploi.ma,Chargé(e) de Service Clients - Mohammedia,BWT MAROC,Casablanca-Mohammedia,BAC+3,CDI,1-2 ans,28.10.2025,"Résumé du poste Commercial, vente Casablanca-M...",https://www.emploi.ma/offre-emploi-maroc/charg...
499,Emploi.ma,Gérant de Station-Service - Ben Maachou,SALMA PLAS,Casablanca-Mohammedia,BAC+3,CDI,Débutant,12.11.2025,"Résumé du poste Commercial, vente Gestion, com...",https://www.emploi.ma/offre-emploi-maroc/geran...


# **Rekrute Scraper**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime, timedelta

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
all_jobs = []
scraped_urls = set()

def normalize_url(url):
    """Normalise l'URL pour éviter les doublons"""
    url = url.split('#')[0].split('?')[0]
    return url.rstrip('/')

def get_job_links(page_number):
    url = f"https://www.rekrute.com/offres.html?s=1&p={page_number}&o=1"
    try:
        r = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
        links = []

        for a in soup.select("a[href*='offre-emploi']"):
            href = a.get("href", "")
            if "offre-emploi" in href:
                if href.startswith("http"):
                    full_url = href
                elif href.startswith("/"):
                    full_url = "https://www.rekrute.com" + href
                else:
                    full_url = "https://www.rekrute.com/" + href

                normalized_url = normalize_url(full_url)

                if normalized_url not in scraped_urls:
                    links.append(normalized_url)
                    scraped_urls.add(normalized_url)

        return links
    except Exception as e:
        print(f"❌ Error fetching page {page_number}: {e}")
        return []

def extract_company(soup):
    """Extraire le nom de l'entreprise"""
    logo_img = soup.find("img", class_="photo")
    if logo_img:
        company_name = logo_img.get("alt") or logo_img.get("title")
        if company_name and company_name.strip():
            return company_name.strip()

    company_link = soup.find("a", href=re.compile(r"-emploi-recrutement-\d+\.html"))
    if company_link:
        href = company_link.get("href", "")
        match = re.search(r'/([a-z0-9-]+)-emploi-recrutement-\d+\.html', href)
        if match:
            company_slug = match.group(1)
            company_name = company_slug.replace('-', ' ').title()
            return company_name

    for div in soup.find_all("div", class_=re.compile(r"company|recruiter")):
        text = div.get_text(strip=True)
        if text and 3 < len(text) < 100:
            if '.' not in text[:50]:
                return text

    return "Confidentiel"

def extract_education_level(soup):
    """Extraire le niveau d'études"""
    education_keywords = {
        'BAC+5': ['bac+5', 'bac +5', 'master', 'ingénieur', 'engineer', 'supérieur'],
        'BAC+4': ['bac+4', 'bac +4', 'licence', 'bachelor'],
        'BAC+3': ['bac+3', 'bac +3', 'technicien', 'dut', 'deug'],
        'BAC+2': ['bac+2', 'bac +2', 'deust', 'ts'],
        'BAC': ['bac', 'baccalauréat'],
        'DOCTORAT': ['doctorat', 'phd', 'doctorant']
    }

    full_text = soup.get_text().lower()

    for level, keywords in education_keywords.items():
        for keyword in keywords:
            if keyword in full_text:
                return level

    for tag in soup.find_all(["li", "span", "div"]):
        text = tag.get_text().lower()
        if any(word in text for word in ['bac', 'niveau', 'formation', 'diplôme', 'étude']):
            for level, keywords in education_keywords.items():
                for keyword in keywords:
                    if keyword in text:
                        return level

    return "Non spécifié"

def extract_contract_type(soup):
    """Extraire le type de contrat"""
    contract_types = {
        'CDI': ['cdi', 'contrat à durée indéterminée'],
        'CDD': ['cdd', 'contrat à durée déterminée'],
        'STAGE': ['stage', 'internship', 'stagiaire'],
        'FREELANCE': ['freelance', 'indépendant', 'consultant'],
        'ANAPEC': ['anapec', 'contrat anapec'],
        'ALTERNANCE': ['alternance', 'apprentissage'],
        'TEMPS PARTIEL': ['temps partiel', 'part time'],
        'MISSION': ['mission', 'contrat de mission']
    }

    full_text = soup.get_text().lower()

    for span in soup.find_all("span", class_=re.compile("tagContract|contrat|type")):
        text = span.get_text().lower()
        for contract, keywords in contract_types.items():
            for keyword in keywords:
                if keyword in text:
                    return contract

    for contract, keywords in contract_types.items():
        for keyword in keywords:
            if keyword in full_text:
                return contract

    return "Non spécifié"

def extract_experience(soup):
    """Extraire l'expérience requise"""
    experience_patterns = [
        (r'(\d+)\s*ans?', lambda x: f"{x} ans"),
        (r'(\d+)\s*à\s*(\d+)\s*ans?', lambda x, y: f"{x}-{y} ans"),
        (r'débutant|junior|entry level', "Débutant"),
        (r'senior|expérimenté|experimente', "Senior"),
        (r'(\d+)\s*années?', lambda x: f"{x} ans"),
        (r'(\d+)\s*ans?', lambda x: f"{x} ans")
    ]

    for tag in soup.find_all(["li", "span", "div"]):
        text = tag.get_text().lower()
        if any(word in text for word in ['expérience', 'experience', 'années', 'ans']):
            for pattern, converter in experience_patterns:
                matches = re.search(pattern, text)
                if matches:
                    if callable(converter):
                        return converter(*matches.groups())
                    else:
                        return converter

    full_text = soup.get_text().lower()
    for pattern, converter in experience_patterns:
        matches = re.search(pattern, full_text)
        if matches:
            if callable(converter):
                return converter(*matches.groups())
            else:
                return converter

    return "Non spécifié"

def extract_publication_date(soup):
    """Extraire la VRAIE date de publication - VERSION FINALE"""

    print("    🔍 Recherche de la date de publication...")

    # METHODE 1: Chercher "Publiée aujourd'hui"
    full_text = soup.get_text()

    if "publiée aujourd'hui" in full_text.lower():
        date_publication = datetime.now().strftime("%d/%m/%Y")
        print(f"    📅 Trouvée: Publiée aujourd'hui → {date_publication}")
        return date_publication

    # METHODE 2: Chercher "Publiée il y a X jours"
    jours_match = re.search(r'publiée il y a (\d+)\s*jours?', full_text, re.IGNORECASE)
    if jours_match:
        jours = int(jours_match.group(1))
        date_publication = (datetime.now() - timedelta(days=jours)).strftime("%d/%m/%Y")
        print(f"    📅 Calculée: Publiée il y a {jours} jours → {date_publication}")
        return date_publication

    # METHODE 3: Chercher "Publiée il y a X heures"
    heures_match = re.search(r'publiée il y a (\d+)\s*heures?', full_text, re.IGNORECASE)
    if heures_match:
        date_publication = datetime.now().strftime("%d/%m/%Y")
        print(f"    📅 Calculée: Publiée il y a {heures_match.group(1)} heures → {date_publication}")
        return date_publication

    # METHODE 4: Chercher dans <em class="date"> qui contient "Publication : du X au Y"
    date_em = soup.find("em", class_="date")
    if date_em:
        em_text = date_em.get_text()
        print(f"    📝 Contenu <em class='date'>: {em_text[:150]}...")

        # Pattern: "Publication : du DD/MM/YYYY au DD/MM/YYYY"
        pub_match = re.search(r'Publication\s*:\s*du\s*(\d{2}/\d{2}/\d{4})\s*au\s*(\d{2}/\d{2}/\d{4})', em_text, re.IGNORECASE)
        if pub_match:
            date_debut = pub_match.group(1)
            date_fin = pub_match.group(2)
            print(f"    📅 Trouvée (format complet): du {date_debut} au {date_fin}")
            print(f"    ✅ Date de publication retenue: {date_debut}")
            return date_debut

        # Pattern: "Publication : du DD/MM/YYYY"
        pub_match_simple = re.search(r'Publication\s*:\s*du\s*(\d{2}/\d{2}/\d{4})', em_text, re.IGNORECASE)
        if pub_match_simple:
            date_publication = pub_match_simple.group(1)
            print(f"    📅 Trouvée (format standard): {date_publication}")
            return date_publication

    # METHODE 5: Chercher "Publication : du DATE au DATE" dans tout le texte
    pub_match_full = re.search(r'Publication\s*:\s*du\s*(\d{2}/\d{2}/\d{4})\s*au\s*(\d{2}/\d{2}/\d{4})', full_text, re.IGNORECASE)
    if pub_match_full:
        date_debut = pub_match_full.group(1)
        date_fin = pub_match_full.group(2)
        print(f"    📅 Trouvée dans texte: du {date_debut} au {date_fin}")
        print(f"    ✅ Date de publication retenue: {date_debut}")
        return date_debut

    # Pattern: "Publication : du DD/MM/YYYY"
    pub_match_simple_full = re.search(r'Publication\s*:\s*du\s*(\d{2}/\d{2}/\d{4})', full_text, re.IGNORECASE)
    if pub_match_simple_full:
        date_publication = pub_match_simple_full.group(1)
        print(f"    📅 Trouvée dans texte: {date_publication}")
        return date_publication

    # METHODE 3: Chercher dans toutes les balises <span> pour les dates
    # Le XPath //*[@id="175288"]/div/div[2]/div/div/em/span[1] suggère span[1]
    all_spans = soup.find_all("span")
    for span in all_spans:
        span_text = span.get_text()
        # Chercher une date DD/MM/YYYY dans ce span
        date_match = re.search(r'(\d{2}/\d{2}/\d{4})', span_text)
        if date_match:
            date_found = date_match.group(1)
            # Vérifier si c'est une date valide (dans le passé récent)
            try:
                date_obj = datetime.strptime(date_found, "%d/%m/%Y")
                today = datetime.now()
                days_diff = (today - date_obj).days

                if 0 <= days_diff <= 180:  # Dans les 6 derniers mois
                    # Vérifier le contexte pour confirmer que c'est bien la date de publication
                    parent_text = span.parent.get_text() if span.parent else ""
                    if "publication" in parent_text.lower() or "publi" in parent_text.lower():
                        print(f"    📅 Trouvée dans <span> (contexte publication): {date_found}")
                        return date_found
            except:
                continue
    date_em = soup.find("em", class_="date")
    if date_em:
        em_text = date_em.get_text()
        print(f"    📝 Contenu <em class='date'>: {em_text}")

        # Extraire "du XX/XX/XXXX"
        date_match = re.search(r'du\s*(\d{2}/\d{2}/\d{4})', em_text)
        if date_match:
            date_publication = date_match.group(1)
            print(f"    📅 Trouvée dans <em>: {date_publication}")
            return date_publication

    # METHODE 4: Chercher "Publiée il y a X jours" et CALCULER la date
    jours_match = re.search(r'publiée il y a (\d+)\s*jours?', full_text, re.IGNORECASE)
    if jours_match:
        jours = int(jours_match.group(1))
        date_publication = (datetime.now() - timedelta(days=jours)).strftime("%d/%m/%Y")
        print(f"    📅 Calculée: Publiée il y a {jours} jours → {date_publication}")
        return date_publication

    # METHODE 5: Chercher "Publiée il y a X heures"
    heures_match = re.search(r'publiée il y a (\d+)\s*heures?', full_text, re.IGNORECASE)
    if heures_match:
        date_publication = datetime.now().strftime("%d/%m/%Y")
        print(f"    📅 Calculée: Publiée aujourd'hui")
        return date_publication

    # METHODE 6: Chercher directement les dates au format DD/MM/YYYY
    # IMPORTANT: On cherche la date de DÉBUT, pas la date limite
    dates = re.findall(r'\b(\d{2}/\d{2}/\d{4})\b', full_text)
    if dates:
        today = datetime.now()
        valid_dates = []

        for date_str in dates:
            try:
                date_obj = datetime.strptime(date_str, "%d/%m/%Y")
                days_diff = (today - date_obj).days

                # Critères de validation :
                # - Date dans le passé (days_diff >= 0)
                # - Pas trop ancienne (moins de 6 mois = 180 jours)
                # - Pas dans le futur
                if 0 <= days_diff <= 180:
                    valid_dates.append((date_str, days_diff))
                    print(f"    ✓ Date valide trouvée: {date_str} (il y a {days_diff} jours)")
            except:
                continue

        # Si on a des dates valides, prendre la PLUS RÉCENTE (la plus proche d'aujourd'hui)
        if valid_dates:
            # Trier par nombre de jours (du plus récent au plus ancien)
            valid_dates.sort(key=lambda x: x[1])
            best_date = valid_dates[0][0]
            print(f"    📅 Date de publication retenue: {best_date}")
            return best_date

        # Si aucune date valide, indiquer clairement qu'on ne peut pas déterminer
        print(f"    ⚠️  Dates trouvées ({len(dates)}) mais aucune dans le passé récent")
        print(f"    Dates détectées: {dates[:3]}")  # Afficher les 3 premières

    print("    ❌ Aucune information de date trouvée")
    return "Non spécifiée"

def extract_city(soup):
    """Extraire la ville"""
    title_tag = soup.select_one("h1")
    if title_tag:
        title = title_tag.text.strip()
        if " - " in title:
            parts = title.split(" - ")
            if len(parts) > 1:
                city_candidate = parts[-1].strip()
                villes_maroc = ['casablanca', 'rabat', 'marrakech', 'tanger', 'fès', 'meknès',
                               'agadir', 'kenitra', 'oujda', 'tetouan', 'salé', 'mohammedia',
                               'el jadida', 'safi', 'beni mellal', 'nador', 'tétouan']
                if any(ville in city_candidate.lower() for ville in villes_maroc):
                    return city_candidate

    city_selectors = [
        "span.city",
        "div.location",
        "li.location",
        "span[title*='Region']",
        "span[title*='Ville']"
    ]

    for selector in city_selectors:
        city_tag = soup.select_one(selector)
        if city_tag:
            city = city_tag.text.strip()
            if city and city != "Non spécifiée":
                return city

    return "Non spécifiée"

def extract_description(soup):
    """Extraire la description"""
    selectors = [
        "div.post-content",
        "div.description",
        "div.job-description",
        "div.details-post",
        "div#recruiterDescription p"
    ]

    for selector in selectors:
        desc = soup.select_one(selector)
        if desc:
            text = desc.get_text(strip=True)
            if len(text) > 50:
                text = re.sub(r'\s+', ' ', text)
                return text[:500]

    return "Description non disponible"

def scrape_job_details(url):
    try:
        print(f"    🔍 Scraping: {url}")
        r = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")

        # Title
        title_tag = soup.select_one("h1")
        title = title_tag.text.strip() if title_tag else "N/A"

        # Company
        company = extract_company(soup)

        # City
        city = extract_city(soup)

        # Niveau d'études
        education = extract_education_level(soup)

        # Type de contrat
        contract = extract_contract_type(soup)

        # Expérience
        experience = extract_experience(soup)

        # Date de publication - CORRIGÉ
        publication = extract_publication_date(soup)

        # Description
        description = extract_description(soup)

        job_data = {
            "site": "Rekrute",
            "title": title,
            "company": company,
            "city": city,
            "niveau_etudes": education,
            "contract": contract,
            "experience": experience,
            "publication_date": publication,
            "description": description,
            "url": url
        }

        return job_data

    except Exception as e:
        print(f"❌ Error scraping {url}: {e}")
        return None

# Main scraping loop
print("🚀 Starting Rekrute scraper...\n")
for page in range(1, 80):
    print(f"📄 Scraping page {page}...")
    links = get_job_links(page)
    print(f"🔗 Found {len(links)} new job links")

    for link in links:
        job = scrape_job_details(link)
        if job:
            all_jobs.append(job)
            print(f"  ✅ {job['company']} - {job['title'][:40]}...")
            print(f"     📅 {job['publication_date']} | 📍 {job['city']}")
        print("-" * 60)

# Save to CSV
if all_jobs:
    df = pd.DataFrame(all_jobs)

    # Supprimer les doublons
    df['url_normalized'] = df['url'].apply(normalize_url)
    df = df.drop_duplicates(subset=['url_normalized'], keep='first')
    df = df.drop(columns=['url_normalized'])

    # Renommer les colonnes en français
    df = df.rename(columns={
        "site": "site",
        "title": "intitulé_poste",
        "company": "entreprise",
        "city": "ville",
        "niveau_etudes": "niveau_études",
        "contract": "type_contrat",
        "experience": "expérience",
        "publication_date": "publication_date",
        "description": "description",
        "url": "lien_offre"
    })

    # Sauvegarder
    df.to_csv("rekrute_jobs_final.csv", index=False, encoding="utf-8-sig")

    print(f"\n🎉 COLLECTE TERMINÉE!")
    print(f"📊 Total offres collectées: {len(df)}")

    # Rapport détaillé
    print(f"\n📈 RAPPORT STATISTIQUE:")
    print(f"• Entreprises: {df['entreprise'].value_counts().head(5).to_dict()}")
    print(f"• Contrats: {df['type_contrat'].value_counts().to_dict()}")
    print(f"• Niveaux études: {df['niveau_études'].value_counts().to_dict()}")
    print(f"• Villes: {df['ville'].value_counts().head(5).to_dict()}")
    print(f"• Dates publication: {df['publication_date'].value_counts().head(10).to_dict()}")

else:
    print("❌ Aucune offre trouvée")

🚀 Starting Rekrute scraper...

📄 Scraping page 1...
🔗 Found 10 new job links
    🔍 Scraping: https://www.rekrute.com/offre-emploi-coordinateur-paie-recrutement-figeac-aero-maroc-nouaceur-177080.html
    🔍 Recherche de la date de publication...
    📅 Trouvée: Publiée aujourd'hui → 13/11/2025
  ✅ Figeac Aero Maroc - Coordinateur Paie - Nouaceur...
     📅 13/11/2025 | 📍 Non spécifiée
------------------------------------------------------------
    🔍 Scraping: https://www.rekrute.com/offre-emploi-ingenieur-genie-civil-conduite-de-chantier-industriel-recrutement-bab-nakhil-tanger-zone-franche-177078.html
    🔍 Recherche de la date de publication...
    📅 Trouvée: Publiée aujourd'hui → 13/11/2025
  ✅ Bab Nakhil - Ingénieur Génie Civil – Conduite de Chan...
     📅 13/11/2025 | 📍 Tanger – Zone Franche
------------------------------------------------------------
    🔍 Scraping: https://www.rekrute.com/offre-emploi-concepteur-aeronautique-senior-recrutement-akkodis-maroc-tanger-177079.html
    🔍

In [ ]:
df.head()

,site,intitulé_poste,entreprise,ville,niveau_études,type_contrat,expérience,publication_date,description,lien_offre
0,Rekrute,Coordinateur Paie - Nouaceur,Figeac Aero Maroc,Non spécifiée,BAC+5,CDI,5 ans,13/11/2025,"FIGEAC AERO est créé en 1989, Spécialisée dans...",https://www.rekrute.com/offre-emploi-coordinat...
1,Rekrute,Ingénieur Génie Civil – Conduite de Chantier I...,Bab Nakhil,Tanger – Zone Franche,BAC+5,CDI,5 ans,13/11/2025,Notresociété de promotion immobilière Bab Nakh...,https://www.rekrute.com/offre-emploi-ingenieur...
2,Rekrute,Concepteur Aéronautique Senior - Tanger,Akkodis Maroc,Tanger,BAC+5,CDI,10 ans,13/11/2025,Akkodis est un leader mondial sur le marché de...,https://www.rekrute.com/offre-emploi-concepteu...
3,Rekrute,Employability Data Analyst - Benguerir,image description,Non spécifiée,BAC+5,CDI,10 ans,13/11/2025,Mohammed VI Polytechnic University is an insti...,https://www.rekrute.com/offre-emploi-employabi...
4,Rekrute,Technicien Metrologue - Benguerir,image description,Non spécifiée,BAC+3,CDI,5 ans,13/11/2025,Mohammed VI Polytechnic University is an insti...,https://www.rekrute.com/offre-emploi-technicie...


In [ ]:
df.tail()

,site,intitulé_poste,entreprise,ville,niveau_études,type_contrat,expérience,publication_date,description,lien_offre
784,Rekrute,Attaché Commercial à Nassim - Casablanca,image description,Casablanca,BAC+3,CDI,1 ans,22/10/2025,"Présent depuis 1997 au Maroc,Manpower Groupest...",https://www.rekrute.com/offre-emploi-attache-c...
785,Rekrute,Directeur de Restaurant H/F - Tout le Maroc,image description,Non spécifiée,BAC+3,CDI,10 ans,22/10/2025,McDonald’s Maroc est une entreprise 100% maroc...,https://www.rekrute.com/offre-emploi-directeur...
786,Rekrute,Asesor de videojuegos – Habla hispana - Rabat,image description,Rabat,BAC+2,CDI,1 ans,22/10/2025,Description non disponible,https://www.rekrute.com/offre-emploi-asesor-de...
787,Rekrute,Stagiaire QHSE - IKEA Zenata - Casablanca,image description,Casablanca,BAC+3,STAGE,Débutant,22/10/2025,"IKEAis a global retailer known for affordable,...",https://www.rekrute.com/offre-emploi-stagiaire...
788,Rekrute,Formateur - Benguerir,image description,Non spécifiée,BAC+5,CDI,5 ans,22/10/2025,Mohammed VI Polytechnic University is an insti...,https://www.rekrute.com/offre-emploi-formateur...


# **Data Collection**

In [ ]:
import pandas as pd
import json
from datetime import datetime

def simple_merge_csv_to_json():
    """Version ultra-simplifiée pour fusionner deux CSV en JSON"""

    # Lire les deux fichiers CSV
    df_rekrute = pd.read_csv("rekrute_jobs_final.csv", encoding='utf-8-sig')
    df_emploi_ma = pd.read_csv("emploi_ma_toutes_offres.csv", encoding='utf-8-sig')

    # Ajouter la colonne 'site' si elle n'existe pas
    if 'site' not in df_rekrute.columns:
        df_rekrute['site'] = 'ReKrute'
    if 'site' not in df_emploi_ma.columns:
        df_emploi_ma['site'] = 'Emploi_ma'

    # Fusionner les DataFrames
    df_combined = pd.concat([df_rekrute, df_emploi_ma], ignore_index=True)

    # Créer la structure JSON
    output_data = {
        "metadata": {
            "date_export": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "total_offres": len(df_combined),
            "sites_scrapes": ["ReKrute", "Emploi_ma"]
        },
        "offres_emploi": df_combined.to_dict('records')
    }

    # Sauvegarder en JSON
    with open("tous_les_jobs_maroc.json", 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)

    print(f"✅ Fusion réussie! {len(df_combined)} offres sauvegardées dans tous_les_jobs_maroc.json")

# Exécuter
if __name__ == "__main__":
    simple_merge_csv_to_json()

✅ Fusion réussie! 1289 offres sauvegardées dans tous_les_jobs_maroc.json


In [ ]:
import pandas as pd
import json
from datetime import datetime
import os

def mount_google_drive():
    """Monter Google Drive"""
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive monté avec succès!")

def simple_merge_csv_to_json_with_drive():
    """Fusionner CSV et sauvegarder dans Google Drive"""

    # Monter Google Drive
    mount_google_drive()

    # Lire les deux fichiers CSV
    df_rekrute = pd.read_csv("rekrute_jobs_final.csv", encoding='utf-8-sig')
    df_emploi_ma = pd.read_csv("emploi_ma_toutes_offres.csv", encoding='utf-8-sig')

    # Ajouter la colonne 'site' si elle n'existe pas
    if 'site' not in df_rekrute.columns:
        df_rekrute['site'] = 'ReKrute'
    if 'site' not in df_emploi_ma.columns:
        df_emploi_ma['site'] = 'Emploi_ma'

    # Fusionner les DataFrames
    df_combined = pd.concat([df_rekrute, df_emploi_ma], ignore_index=True)

    # Créer la structure JSON
    output_data = {
        "metadata": {
            "date_export": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "total_offres": len(df_combined),
            "sites_scrapes": ["ReKrute", "Emploi_ma"]
        },
        "offres_emploi": df_combined.to_dict('records')
    }

    # Sauvegarder en JSON localement
    local_filename = "tous_les_jobs_maroc.json"
    with open(local_filename, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)

    # Sauvegarder dans Google Drive
    drive_path = "/content/drive/MyDrive/tous_les_jobs_maroc.json"
    with open(drive_path, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)

    print(f"✅ Fusion réussie! {len(df_combined)} offres sauvegardées")
    print(f"📁 Fichier local: {local_filename}")
    print(f"📁 Fichier Drive: {drive_path}")

# Fonction pour afficher les données
def display_json_data(json_file="tous_les_jobs_maroc.json"):
    # Charger le JSON
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Afficher les métadonnées
    print("📊 MÉTADONNÉES:")
    print(f"• Date d'export: {data['metadata']['date_export']}")
    print(f"• Total offres: {data['metadata']['total_offres']}")
    print(f"• Sites scrapés: {', '.join(data['metadata']['sites_scrapes'])}")

    # Convertir en DataFrame pour un affichage tabulaire
    df = pd.DataFrame(data['offres_emploi'])

    print("\n" + "="*80)
    print("👀 APERÇU DES DONNÉES (équivalent de df.head()):")
    print("="*80)

    # Afficher les premières lignes
    print(f"\n📋 Premières {min(10, len(df))} offres:")
    print(df.head(10).to_string(max_colwidth=30))

    return df

# Exécuter
if __name__ == "__main__":
    simple_merge_csv_to_json_with_drive()
    print("\n" + "="*50)
    df = display_json_data()

Mounted at /content/drive
✅ Google Drive monté avec succès!
✅ Fusion réussie! 1289 offres sauvegardées
📁 Fichier local: tous_les_jobs_maroc.json
📁 Fichier Drive: /content/drive/MyDrive/tous_les_jobs_maroc.json

📊 MÉTADONNÉES:
• Date d'export: 2025-11-13 19:15:36
• Total offres: 1289
• Sites scrapés: ReKrute, Emploi_ma

👀 APERÇU DES DONNÉES (équivalent de df.head()):

📋 Premières 10 offres:
      site                 intitulé_poste         entreprise                  ville niveau_études type_contrat expérience publication_date                    description                     lien_offre
0  Rekrute   Coordinateur Paie - Nouaceur  Figeac Aero Maroc          Non spécifiée         BAC+5          CDI      5 ans       13/11/2025  FIGEAC AERO est créé en 19...  https://www.rekrute.com/of...
1  Rekrute  Ingénieur Génie Civil – Co...         Bab Nakhil  Tanger – Zone Franche         BAC+5          CDI      5 ans       13/11/2025  Notresociété de promotion ...  https://www.rekrute.com/of...
2  R

# **PRÉTRAITEMENT**

**Prétraitement Simple**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import pandas as pd

# Charger le JSON
file_path = "/content/drive/MyDrive/tous_les_jobs_maroc.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Transformer en DataFrame
df = pd.json_normalize(data["offres_emploi"])

# Inspecter les données
print(df.head())
print(df.info())

      site                                     intitulé_poste  \
0  Rekrute                       Coordinateur Paie - Nouaceur   
1  Rekrute  Ingénieur Génie Civil – Conduite de Chantier I...   
2  Rekrute            Concepteur Aéronautique Senior - Tanger   
3  Rekrute             Employability Data Analyst - Benguerir   
4  Rekrute                  Technicien Metrologue - Benguerir   

          entreprise                  ville niveau_études type_contrat  \
0  Figeac Aero Maroc          Non spécifiée         BAC+5          CDI   
1         Bab Nakhil  Tanger – Zone Franche         BAC+5          CDI   
2      Akkodis Maroc                 Tanger         BAC+5          CDI   
3  image description          Non spécifiée         BAC+5          CDI   
4  image description          Non spécifiée         BAC+3          CDI   

  expérience publication_date  \
0      5 ans       13/11/2025   
1      5 ans       13/11/2025   
2     10 ans       13/11/2025   
3     10 ans       13/11/2025   

In [ ]:
import re

# Combiner titre + description
df["text"] = df["intitulé_poste"].fillna("") + " " + df["description"].fillna("")

# Nettoyage léger
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)      # URLs
    text = re.sub(r'\S+@\S+', '', text)             # emails
    text = re.sub(r'[^\w\s\-\+\.,;:àâçéèêëîïôûùüÿñæœ]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()       # espaces multiples
    return text

df["clean_text"] = df["text"].apply(clean_text)

# Supprimer les doublons et les valeurs manquantes
df.dropna(subset=["clean_text"], inplace=True)
df.drop_duplicates(subset=["clean_text"], inplace=True)

In [ ]:
# Inspecter les données
print(df.head())
print(df.info())

      site                                     intitulé_poste  \
0  Rekrute                       Coordinateur Paie - Nouaceur   
1  Rekrute  Ingénieur Génie Civil – Conduite de Chantier I...   
2  Rekrute            Concepteur Aéronautique Senior - Tanger   
3  Rekrute             Employability Data Analyst - Benguerir   
4  Rekrute                  Technicien Metrologue - Benguerir   

          entreprise                  ville niveau_études type_contrat  \
0  Figeac Aero Maroc          Non spécifiée         BAC+5          CDI   
1         Bab Nakhil  Tanger – Zone Franche         BAC+5          CDI   
2      Akkodis Maroc                 Tanger         BAC+5          CDI   
3  image description          Non spécifiée         BAC+5          CDI   
4  image description          Non spécifiée         BAC+3          CDI   

  expérience publication_date  \
0      5 ans       13/11/2025   
1      5 ans       13/11/2025   
2     10 ans       13/11/2025   
3     10 ans       13/11/2025   

**Prétraitement total**

IMPORTANT : on ne touche jamais aux colonnes utiles pour l’affichage (site, lien_offre, ville, entreprise). On ne prétraite que le texte utilisé pour l’entraînement NLP.

➡️On garde les colonnes brutes, on crée une colonne prétraitée.

In [ ]:
# 5 Lemmatisation et suppression stopwords (optionnel pour NLP classique)
#
import spacy
!python -m spacy download fr_core_news_sm
nlp = spacy.load("fr_core_news_sm")

def preprocess_spacy(text):
    doc = nlp(text)
    tokens = []
    for token in doc:      #spaCy découpe le texte en tokens
        if token.is_stop or token.is_punct or token.like_num:   #enlèver les stopwords “et”, “de”, “le”, “la”, “un”…
                 continue
        lemma = token.lemma_.strip()
        if len(lemma) > 2:
            tokens.append(lemma)
    return " ".join(tokens)

df["text_clean"] = df["clean_text"].apply(preprocess_spacy)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 98.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# 6 Garder les colonnes originales intactes
# Colonnes utiles pour l’affichage :
# site, lien_offre, entreprise, ville, type_contrat, niveau_études, publication_date
columns_to_keep = ["site", "intitulé_poste", "entreprise", "ville",
                   "niveau_études", "type_contrat", "expérience",
                   "publication_date", "description", "lien_offre",
                   "text_clean"]

df_final = df[columns_to_keep]

In [ ]:
print(df_final.head())
print(df_final.info())

      site                                     intitulé_poste  \
0  Rekrute                       Coordinateur Paie - Nouaceur   
1  Rekrute  Ingénieur Génie Civil – Conduite de Chantier I...   
2  Rekrute            Concepteur Aéronautique Senior - Tanger   
3  Rekrute             Employability Data Analyst - Benguerir   
4  Rekrute                  Technicien Metrologue - Benguerir   

          entreprise                  ville niveau_études type_contrat  \
0  Figeac Aero Maroc          Non spécifiée         BAC+5          CDI   
1         Bab Nakhil  Tanger – Zone Franche         BAC+5          CDI   
2      Akkodis Maroc                 Tanger         BAC+5          CDI   
3  image description          Non spécifiée         BAC+5          CDI   
4  image description          Non spécifiée         BAC+3          CDI   

  expérience publication_date  \
0      5 ans       13/11/2025   
1      5 ans       13/11/2025   
2     10 ans       13/11/2025   
3     10 ans       13/11/2025   

In [ ]:
print(df["text_clean"].head())

0    coordinateur pai nouaceur figeac aero créer sp...
1    ingénieur génie civil conduite chantier indust...
2    concepteur aéronautique senior tanger akkodi l...
3    employability dater analyst benguerir mohammed...
4    technicien metrologue benguerir mohammed polyt...
Name: text_clean, dtype: object


In [ ]:
output_file_path = "/content/drive/MyDrive/tous_les_jobs_maroc_pretraite.csv"
df_final.to_csv(output_file_path, index=False, encoding='utf-8')
print(f"Fichier enregistré avec succès à : {output_file_path}")

Fichier enregistré avec succès à : /content/drive/MyDrive/tous_les_jobs_maroc_pretraite.csv


# **Traitement**

**1. Installation de bibliothèque SentenceTransformer**

In [ ]:
!pip install -U sentence-transformers

**Importation des librairies nécessaires**

In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

**3. Chargement du CSV prétraité**

In [ ]:
df_final = pd.read_csv("/content/drive/MyDrive/tous_les_jobs_maroc_pretraite.csv")

In [ ]:
df_final[["intitulé_poste", "text_clean"]].head()

,intitulé_poste,text_clean
0,Coordinateur Paie - Nouaceur,coordinateur pai nouaceur figeac aero créer sp...
1,Ingénieur Génie Civil – Conduite de Chantier I...,ingénieur génie civil conduite chantier indust...
2,Concepteur Aéronautique Senior - Tanger,concepteur aéronautique senior tanger akkodi l...
3,Employability Data Analyst - Benguerir,employability dater analyst benguerir mohammed...
4,Technicien Metrologue - Benguerir,technicien metrologue benguerir mohammed polyt...


**3. Chargement de modèle d'embeddings**

In [ ]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**4. Transformation : les descriptions en embeddings**

In [ ]:
embeddings_jobs = model.encode(
    df_final["text_clean"].tolist(),
    convert_to_tensor=False,   # numpy array
    show_progress_bar=True
)

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

**Sauvegardage des embeddings**

In [ ]:
np.save("/content/drive/MyDrive/embeddings_jobs.npy", embeddings_jobs)

print("➡️ Embeddings sauvegardés avec succès !")

➡️ Embeddings sauvegardés avec succès !


In [ ]:
print("Shape des embeddings :", np.array(embeddings_jobs).shape)

Shape des embeddings : (1252, 768)


In [ ]:
output_file_path = "/content/drive/MyDrive/tous_les_jobs_maroc_pretraite.csv"
df_final.to_csv(output_file_path, index=False, encoding="utf-8")

print(f"Fichier enregistré avec succès à : {output_file_path}")

Fichier enregistré avec succès à : /content/drive/MyDrive/tous_les_jobs_maroc_pretraite.csv


In [ ]:
!pip install torch

In [ ]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util

# 1) Charger le CSV prétraité
df = pd.read_csv("/content/drive/MyDrive/tous_les_jobs_maroc_pretraite.csv")
print("✔️ Dataset chargé :", df.shape)

# 2) Charger les embeddings sauvegardés
embeddings_jobs = np.load("/content/drive/MyDrive/embeddings_jobs.npy")
print("✔️ Embeddings chargés :", embeddings_jobs.shape)

# 3) Convertir vers Tensor PyTorch
embeddings_jobs_tensor = torch.from_numpy(embeddings_jobs).float()
print("✔️ Conversion en tenseur réussie :", embeddings_jobs_tensor.shape)

✔️ Dataset chargé : (1252, 11)
✔️ Embeddings chargés : (1252, 768)
✔️ Conversion en tenseur réussie : torch.Size([1252, 768])


**Créer de la fonction de filtrage**

In [ ]:
def filter_jobs(df, filters):
    df_filtered = df.copy()

    for key, value in filters.items():
        if value and key in df.columns:
            df_filtered = df_filtered[df_filtered[key].str.contains(value, na=False, case=False)]

    return df_filtered

**Créer la fonction de recommandation TOP-K**

In [ ]:
def recommend_jobs(user_text, top_k=5, filters=None):
    # Embedding requête utilisateur
    user_embedding = model.encode(user_text, convert_to_tensor=True)

    # Similarité cosinus
    scores = util.cos_sim(user_embedding, embeddings_jobs_tensor)[0]

    # Classement décroissant
    top_results = scores.cpu().numpy().argsort()[::-1]

    recommendations = []

    for idx in top_results:
        job = df.iloc[idx]

        # appliquer filtres si demandés
        if filters:
            match_filter = True
            for k, v in filters.items():
                if v and (v.lower() not in str(job[k]).lower()):
                    match_filter = False
                    break
            if not match_filter:
                continue

        recommendations.append({
            "titre": job["intitulé_poste"],
            "entreprise": job["entreprise"],
            "ville": job["ville"],
            "contrat": job["type_contrat"],
            "date": job["publication_date"],
            "lien": job["lien_offre"],
            "score": float(scores[idx])
        })

        if len(recommendations) == top_k:
            break

    return recommendations

**Test**

In [ ]:
query = "développeur python machine learning casablanca"

filters = {
    "ville": "Casablanca",
    "type_contrat": "",
    "niveau_études": ""
}

recs = recommend_jobs(query, top_k=5, filters=filters)

for r in recs:
    print("----")
    print(f"{r['titre']} - {r['entreprise']} ({r['ville']})")
    print(f"Contrat : {r['contrat']} | {r['date']}")
    print(r["lien"])
    print("Score :", r["score"])

----
DevOps Sénior Python - Casablanca - VISION BUSINESS CONSULTING (Casablanca-Mohammedia)
Contrat : FREELANCE | 22.10.2025
https://www.emploi.ma/offre-emploi-maroc/devops-senior-python-casablanca-9073979
Score : 0.5826689004898071
----
Administrateur ERP - Sage X3 - Casablanca - COLORADO (Casablanca-Mohammedia)
Contrat : CDI | 12.11.2025
https://www.emploi.ma/offre-emploi-maroc/administrateur-erp-sage-x3-casablanca-9060481
Score : 0.5356804132461548
----
Concepteur Développeur Python (H/F) - Casablanca - image description (Casablanca)
Contrat : CDI | 13/11/2025
https://www.rekrute.com/offre-emploi-concepteur-developpeur-python-hf-recrutement-sofrecom-maroc-casablanca-177034.html
Score : 0.5200065970420837
----
Chef de Projet Sage X3 - Casablanca - COLORADO (Casablanca-Mohammedia)
Contrat : CDI | 10.11.2025
https://www.emploi.ma/offre-emploi-maroc/chef-projet-sage-x3-casablanca-8840064
Score : 0.5014314651489258
----
Ingénieur Test et Automatisation - Casablanca - VISION BUSINESS CONS